In [0]:
# -*- coding: utf-8 -*-
from datetime import datetime, timedelta
from __future__ import print_function

import pandas.io.data as web
import statsmodels.api as sm
import pandas as pd
import numpy as np
import pymysql

conn = pymysql.connect(host='', port=3306, user='', passwd='', db='')

cur = conn.cursor()
dataframe = pd.DataFrame()

In [0]:
class Fund_Bond_simulation:
    def __init__(self, code, start_date, end_date):
        self.code = code
        self.start_date = start_date
        self.end_date = end_date
    
    def DB_connect(self):
        conn = pymysql.connect(host='', port=3306, user='', passwd='', db='')
        return conn
    
    def get_data(self):
        start_date = self.start_date
        end_date = self.end_date
        code = self.code
        date_list = []
        price_list = []
        for i in range(0,len(code)):
            cur.execute("SELECT date, standardPrice from fund_price where code = '%s' and date >= '%s' and  date <= '%s' order by date ASC"
                       %(code[i], start_date, end_date))
            for row in cur:
                date_list.append(row[0])
                price_list.append(row[1])
            if(i == 0):
                dataframe['Date'] = date_list
            
            dataframe[code[i]] = price_list
            del date_list[:]
            del price_list[:]
            
        return dataframe

    #전월 대비 수익률
    def get_ret_data(self): 
        data = self.get_data().convert_objects(convert_numeric=True)
        for i in self.code:     
            dataframe[i + '_ret'] = (data[i] / data[i][0]) 
        return dataframe
    
    def rebalance(self):
        data1 = self.get_ret_data().convert_objects(convert_numeric=True)
        code = self.code
        balance_list = []
        balance_list.append(1.0)
        
        for i in range(1,len(data1)):
            balance_list.append(round((data1[code[0]+'_ret'][i] + data1[code[1]+'_ret'][i]) / 2,3) * round(balance_list[i-1],3))
            print('================================')
            print(round((data1[code[0]+'_ret'][i] + data1[code[1]+'_ret'][i]) / 2,3))
            print(round(balance_list[i-1],3))
            print(round((data1[code[0]+'_ret'][i] + data1[code[1]+'_ret'][i]) / 2,3) * round(balance_list[i-1],3))
        dataframe['rebalance'] = balance_list
        print(balance_list)
        return dataframe
    
    #이동평균선 매매 전략
    def moving_avg_strategy(self):
        data2 = self.get_data().convert_objects(convert_numeric=True)
        code = self.code
        data2['MA20'] = pd.stats.moments.rolling_mean(data2[code[0]],20) #20일 이평선
        
        first_index_list = [] 
        #중,장기 이동평균선의 시작점을 찾는 과정
        for i in data2.index:
            if pd.isnull(data2.ix[i]['MA20']):
                first_index_list.append(i)
        first_index = first_index_list[-1] + 1
        print(first_index)
        
        accumulate_return = [] #누적 수익률
        first_close = 0 #처음 종가(단기 이동 평균선 > 중,장기 이동 평균선) 
        end_close = 0 #마지막 종가(단기 이동 평균선 == 중,장기 이동 평균선) 
        status = 1 #주식을 살때 상태, 0이면 파는 상태
        sell_status = [] #사고 팔기 상태
        index = 1
        
        for i in range(first_index, len(data2.index)):
            if index == 1:
                if data2.ix[i][code[0]] > data2.ix[i]['MA20']:
                    first_close = data2.ix[i][code[0]]
                    print('산다')
                    sell_status.append('산다'.decode('utf-8'))
                    print('살때 가격: '+str(first_close))
                    print(data2.ix[i]['Date'], data2.ix[i]['MA20'])
                    index = 0
            elif index == 0:
                if data2.ix[i][code[0]] < data2.ix[i]['MA20']:
                    end_close = data2.ix[i][code[0]]
                    print(end_close)
                    accumulate_return.append(((end_close - first_close) / first_close) + 1)
                    print('판다')
                    sell_status.append('판다')
                    print('팔때 가격: '+str(first_close))
                    print(data2.ix[i]['Date'], data2.ix[i]['MA20'])
                    index = 1 
                    
                else:
                    print('유지')
                    sell_status.append('유지')
                    print(data2.ix[i]['Date'], data2.ix[i]['MA20'])
        print(sell_status[0].encode('utf-8'))
        print(len(sell_status))
        #data2['accumulate_return'][first_index:] = accumulate_return
        return data2
        
fund_code_list = []  
Fund_Code = raw_input('Fund_Code: ')
Bond_Code = raw_input('Bond_Code: ')
start_date = str(raw_input('date'))
end_date = str(raw_input('date'))

if Fund_Code:
    fund_code_list.append(Fund_Code)
elif Bond_Code:
    fund_code_list.append(Bond_Code)
else:
    print('The code is empty')

b = Fund_Bond_simulation(fund_code_list, start_date, end_date)
print(b.moving_avg_strategy())

Fund_Code: KR5207579487
Bond_Code: 
date2015.01.01
date2016.01.01
19
산다
살때 가격: 963.27
2015.01.29 937.2665
유지
2015.01.30 938.649
유지
2015.02.02 939.87
유지
2015.02.03 941.0785
유지
2015.02.04 943.081
유지
2015.02.05 945.48
유지
2015.02.06 947.1285
유지
2015.02.09 948.4955
유지
2015.02.10 949.65
950.28
판다
팔때 가격: 963.27
2015.02.11 950.419
산다
살때 가격: 953.07
2015.02.12 951.3695
950.02
판다
팔때 가격: 953.07
2015.02.13 952.13
산다
살때 가격: 958.07
2015.02.16 953.665
953.8
판다
팔때 가격: 958.07
2015.02.17 954.729
산다
살때 가격: 957.37
2015.02.24 955.9405
유지
2015.02.25 956.877
유지
2015.02.26 957.595
유지
2015.02.27 958.5245
유지
2015.03.02 958.859
유지
2015.03.03 959.346
유지
2015.03.04 960.004
유지
2015.03.05 960.6335
유지
2015.03.06 961.178
유지
2015.03.09 962.1015
유지
2015.03.10 962.161
961.18
판다
팔때 가격: 957.37
2015.03.11 962.245
산다
살때 가격: 962.87
2015.03.12 962.28
957.49
판다
팔때 가격: 962.87
2015.03.13 962.2915
산다
살때 가격: 963.08
2015.03.16 962.9315
유지
2015.03.17 963.512
유지
2015.03.18 965.1175
유지
2015.03.19 966.2755
유지
2015.03.20 967.8875
유지
2015.

C:\Users\haklim\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\haklim\Anaconda2\lib\site-packages\ipykernel\__main__.py:71: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=20,center=False).mean()
